In [1]:
from mxnet import nd
from mxnet.contrib import text
text.embedding.get_pretrained_file_names().keys()

dict_keys(['fasttext', 'glove'])

In [2]:
fnames = text.embedding.get_pretrained_file_names('glove')
print(fnames)

['glove.twitter.27B.50d.txt', 'glove.twitter.27B.100d.txt', 'glove.twitter.27B.200d.txt', 'glove.6B.300d.txt', 'glove.6B.50d.txt', 'glove.42B.300d.txt', 'glove.twitter.27B.25d.txt', 'glove.6B.100d.txt', 'glove.6B.200d.txt', 'glove.840B.300d.txt']


In [3]:
glove_6B50d = text.embedding.create('glove', pretrained_file_name = fnames[4])

## Finding Synonyms

#### Define function

In [57]:
def knn(W, x, k):
    cos = nd.dot(W, x.reshape((-1,))) / ((nd.sum(W*W, axis = 1) + 1e-7).sqrt() * nd.sum(x*x).sqrt())
    topk = nd.topk(cos, k = k, ret_typ='indices').asnumpy().astype('int32')
    return topk, [cos[i].asscalar() for i in topk]

In [63]:
def get_similar_tokens(query, k, embed):
    topk, cos = knn(embed.idx_to_vec,
                   embed.get_vecs_by_tokens([query]), k+1)
    for i, c in zip(topk[1:], cos[1:]):
        print('cosine similarity = %.3f: %s' %(c, embed.idx_to_token[i]))

#### Find similar words

In [64]:
get_similar_tokens('starcraft', 3, glove_6B50d)

cosine similarity = 0.802: warcraft
cosine similarity = 0.775: shenmue
cosine similarity = 0.739: everquest


In [65]:
get_similar_tokens('neural', 4, glove_6B50d)

cosine similarity = 0.845: neuronal
cosine similarity = 0.805: differentiation
cosine similarity = 0.782: neurons
cosine similarity = 0.781: pathways


## Finding Analogies

#### Define function

In [66]:
def get_analogy(token_analogy_1, token_analogy_2, token_target, embed):
    vecs = embed.get_vecs_by_tokens([token_analogy_1, token_analogy_2, token_target])
    x = vecs[1] - vecs[0] + vecs[2]
    pred, cos = knn(embed.idx_to_vec, x, 1)
    return embed.idx_to_token[pred[0]]

#### Find anologies

In [67]:
get_analogy('man', 'woman', 'son', glove_6B50d)

'daughter'

In [71]:
get_analogy('england', 'london', 'korea', glove_6B50d)

'seoul'

In [82]:
get_analogy('do', 'did', 'go', glove_6B50d)

'went'